# Yhaoo! Finace Scrapper

In [131]:
import re
import json
import csv
from io import StringIO
from bs4 import BeautifulSoup
from selenium import webdriver
import requests


In [132]:
driver=webdriver.Chrome(executable_path="F:\Web_Scraping\chromedriver.exe")

In [133]:
url_status='https://in.finance.yahoo.com/quote/{}/key-statistics?p={}'
url_profile='https://in.finance.yahoo.com/quote/{}/profile?p={}'
url_finacials='https://in.finance.yahoo.com/quote/{}/financials?p={}'

In [134]:
stock='F'

In [135]:
response= requests.get(url_finacials.format(stock,stock))

In [136]:
soup= BeautifulSoup(response.text,'html.parser')

In [137]:
pattern= re.compile(r'\s--\sData\s--\s')
script_data=soup.find('script',text=pattern).contents[0]

In [138]:
print(script_data[:500])
print('end',script_data[-500:])


(function (root) {
/* -- Data -- */
root.App || (root.App = {});
root.App.now = 1605382436289;
root.App.main = {"context":{"dispatcher":{"stores":{"PageStore":{"currentPageName":"quote","currentEvent":{"eventName":"NEW_PAGE_SUCCESS"},"currentRenderTargetId":"default","pagesConfigRaw":{"base":{"quote":{"layout":{"bundleName":"yahoodotcom-layout.TwoColumnLayout","name":"TwoColumnLayout","config":{"enableHeaderCollapse":true,"additionalBodyWrapperClasses":"Bgc($layoutBgColor)!","contentWrapperClas
end lideshow":{"strings":1},"tdv2-applet-sponsored-moments":{"strings":1},"tdv2-wafer-utils":{"strings":1},"tdv2-applet-stream":{"strings":1},"tdv2-applet-stream-hero":{"strings":1},"tdv2-applet-swisschamp":{"strings":1},"tdv2-applet-uh":{"strings":1},"tdv2-applet-userintent":{"strings":1},"tdv2-applet-video-lightbox":{"strings":1},"tdv2-applet-video-modal":{"strings":1},"tdv2-wafer-header":{"strings":1},"yahoodotcom-layout":{"strings":1}}},"options":{"defaultBundle":"td-app-finance"}}}};
}(thi

In [139]:
start =script_data.find("context")-2

In [140]:
start

112

In [141]:
json_data=json.loads(script_data[start:-12])

In [142]:
json_data['context'].keys()

dict_keys(['dispatcher', 'options', 'plugins'])

In [143]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()

dict_keys(['financialsTemplate', 'cashflowStatementHistory', 'balanceSheetHistoryQuarterly', 'earnings', 'price', 'incomeStatementHistoryQuarterly', 'incomeStatementHistory', 'balanceSheetHistory', 'cashflowStatementHistoryQuarterly', 'quoteType', 'summaryDetail', 'symbol', 'pageViews'])

In [144]:
annual_is=json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistory']['incomeStatementHistory']
quertly_is=json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistoryQuarterly']['incomeStatementHistory']

anunual_cf=json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistory']['cashflowStatements']
quertly_cf=json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistoryQuarterly']['cashflowStatements']

annual_bs=json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistory']['balanceSheetStatements']
quertly_bs=json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistoryQuarterly']['balanceSheetStatements']

In [145]:
print(annual_is[0])

{'researchDevelopment': {}, 'effectOfAccountingCharges': {}, 'incomeBeforeTax': {'raw': -640000000, 'fmt': '-640M', 'longFmt': '-640,000,000'}, 'minorityInterest': {'raw': 45000000, 'fmt': '45M', 'longFmt': '45,000,000'}, 'netIncome': {'raw': 47000000, 'fmt': '47M', 'longFmt': '47,000,000'}, 'sellingGeneralAdministrative': {'raw': 10218000000, 'fmt': '10.22B', 'longFmt': '10,218,000,000'}, 'grossProfit': {'raw': 12876000000, 'fmt': '12.88B', 'longFmt': '12,876,000,000'}, 'ebit': {'raw': 2658000000, 'fmt': '2.66B', 'longFmt': '2,658,000,000'}, 'endDate': {'raw': 1577750400, 'fmt': '2019-12-31'}, 'operatingIncome': {'raw': 2658000000, 'fmt': '2.66B', 'longFmt': '2,658,000,000'}, 'otherOperatingExpenses': {}, 'interestExpense': {'raw': -1049000000, 'fmt': '-1.05B', 'longFmt': '-1,049,000,000'}, 'extraordinaryItems': {}, 'nonRecurring': {}, 'otherItems': {}, 'incomeTaxExpense': {'raw': -724000000, 'fmt': '-724M', 'longFmt': '-724,000,000'}, 'totalRevenue': {'raw': 155900000000, 'fmt': '155

In [146]:
annual_is[0]['operatingIncome']

{'raw': 2658000000, 'fmt': '2.66B', 'longFmt': '2,658,000,000'}

In [147]:
annual_is_stmts=[]
qutarly_is_stmts=[]
# consolidate annual

for s in annual_is:
    statement={}
    for key,val in s.items():
        try:
            statement[key]=val['raw']
        except TypeError:
            continue
        except KeyError:
            continue
    annual_is_stmts.append(statement)
    

# consolidate quertly

for s in quertly_is:
    statement={}
    for key,val in s.items():
        try:
            statement[key]=val['raw']
        except TypeError:
            continue
        except KeyError:
            continue
    qutarly_is_stmts.append(statement)

In [148]:
annual_is_stmts[0]

{'incomeBeforeTax': -640000000,
 'minorityInterest': 45000000,
 'netIncome': 47000000,
 'sellingGeneralAdministrative': 10218000000,
 'grossProfit': 12876000000,
 'ebit': 2658000000,
 'endDate': 1577750400,
 'operatingIncome': 2658000000,
 'interestExpense': -1049000000,
 'incomeTaxExpense': -724000000,
 'totalRevenue': 155900000000,
 'totalOperatingExpenses': 153242000000,
 'costOfRevenue': 143024000000,
 'totalOtherIncomeExpenseNet': -3298000000,
 'netIncomeFromContinuingOps': 84000000,
 'netIncomeApplicableToCommonShares': 47000000}

# Profile Data !!

In [149]:
response=requests.get(url_profile.format(stock,stock))
soup= BeautifulSoup(response.text,'html.parser')
pattern= re.compile(r'\s--\sData\s--\s')
script_data=soup.find('script',text=pattern).contents[0]
start =script_data.find("context")-2
json_data=json.loads(script_data[start:-12])

In [150]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()

dict_keys(['financialsTemplate', 'price', 'secFilings', 'quoteType', 'calendarEvents', 'summaryDetail', 'symbol', 'assetProfile', 'pageViews'])

In [151]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile'].keys()


dict_keys(['zip', 'sector', 'fullTimeEmployees', 'compensationRisk', 'auditRisk', 'longBusinessSummary', 'city', 'phone', 'state', 'shareHolderRightsRisk', 'compensationAsOfEpochDate', 'governanceEpochDate', 'boardRisk', 'country', 'companyOfficers', 'website', 'maxAge', 'overallRisk', 'address1', 'industry'])

In [152]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['companyOfficers']

[{'totalPay': {'raw': 3661316, 'fmt': '3.66M', 'longFmt': '3,661,316'},
  'exercisedValue': {'raw': 0, 'fmt': None, 'longFmt': '0'},
  'yearBorn': 1957,
  'name': 'Mr. William Clay Ford Jr.',
  'title': 'Exec. Chairman',
  'maxAge': 1,
  'fiscalYear': 2019,
  'unexercisedValue': {'raw': 0, 'fmt': None, 'longFmt': '0'},
  'age': 62},
 {'totalPay': {'raw': 2274993, 'fmt': '2.27M', 'longFmt': '2,274,993'},
  'exercisedValue': {'raw': 0, 'fmt': None, 'longFmt': '0'},
  'yearBorn': 1962,
  'name': 'Mr. James D. Farley Jr.',
  'title': 'Pres, CEO & Director',
  'maxAge': 1,
  'fiscalYear': 2019,
  'unexercisedValue': {'raw': 0, 'fmt': None, 'longFmt': '0'},
  'age': 57},
 {'totalPay': {'raw': 4167237, 'fmt': '4.17M', 'longFmt': '4,167,237'},
  'exercisedValue': {'raw': 0, 'fmt': None, 'longFmt': '0'},
  'yearBorn': 1955,
  'name': 'Mr. James Patrick Hackett',
  'title': 'Advisor',
  'maxAge': 1,
  'fiscalYear': 2019,
  'unexercisedValue': {'raw': 0, 'fmt': None, 'longFmt': '0'},
  'age': 64}

In [153]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['longBusinessSummary']

'Ford Motor Company designs, manufactures, markets, and services a range of Ford cars, trucks, sport utility vehicles, electrified vehicles, and Lincoln luxury vehicles worldwide. It operates through three segments: Automotive, Mobility, and Ford Credit. The Automotive segment sells Ford and Lincoln vehicles, service parts, and accessories through distributors and dealers, as well as through dealerships to commercial fleet customers, daily rental car companies, and governments. The Mobility segment designs and builds mobility services; and provides self-driving systems development and vehicle integration, autonomous vehicle research and engineering, and autonomous vehicle transportation-as-a-service network development services. The Ford Credit segment primarily engages in vehicle-related financing and leasing activities to and through automotive dealers. It provides retail installment sale contracts for new and used vehicles; and direct financing leases for new vehicles to retail and 

In [154]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['secFilings'].keys()

dict_keys(['filings', 'maxAge'])

In [155]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['secFilings']['filings']

[{'date': '2020-10-29',
  'epochDate': 1603969511,
  'type': '10-Q',
  'title': 'Quarterly Report',
  'edgarUrl': 'https://yahoo.brand.edgar-online.com/DisplayFiling.aspx?TabIndex=2&dcn=0000037996-20-000071&nav=1&src=Yahoo',
  'maxAge': 1},
 {'date': '2020-10-28',
  'epochDate': 1603919714,
  'type': '8-K',
  'title': 'Results of Operations and Financial Condition, Financial Statements and Exhibits',
  'edgarUrl': 'https://yahoo.brand.edgar-online.com/DisplayFiling.aspx?TabIndex=2&dcn=0000037996-20-000070&nav=1&src=Yahoo',
  'maxAge': 1},
 {'date': '2020-10-15',
  'epochDate': 1602769209,
  'type': '8-K',
  'title': 'Change in Directors or Principal Officers, Financial Statements and Exhibits',
  'edgarUrl': 'https://yahoo.brand.edgar-online.com/DisplayFiling.aspx?TabIndex=2&dcn=0000037996-20-000066&nav=1&src=Yahoo',
  'maxAge': 1},
 {'date': '2020-10-02',
  'epochDate': 1601667709,
  'type': '8-K',
  'title': 'Other Events, Financial Statements and Exhibits',
  'edgarUrl': 'https://ya

In [95]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['summaryDetail']

{'previousClose': {'raw': 8.21, 'fmt': '8.21'},
 'regularMarketOpen': {'raw': 8.23, 'fmt': '8.23'},
 'twoHundredDayAverage': {'raw': 6.6803546, 'fmt': '6.68'},
 'trailingAnnualDividendYield': {'raw': 0.036540806, 'fmt': '3.65%'},
 'payoutRatio': {},
 'volume24Hr': {},
 'regularMarketDayHigh': {'raw': 8.57, 'fmt': '8.57'},
 'navPrice': {},
 'averageDailyVolume10Day': {'raw': 76730700,
  'fmt': '76.73M',
  'longFmt': '76,730,700'},
 'totalAssets': {},
 'regularMarketPreviousClose': {'raw': 8.21, 'fmt': '8.21'},
 'fiftyDayAverage': {'raw': 7.6033335, 'fmt': '7.60'},
 'trailingAnnualDividendRate': {'raw': 0.3, 'fmt': '0.30'},
 'open': {'raw': 8.23, 'fmt': '8.23'},
 'toCurrency': None,
 'averageVolume10days': {'raw': 76730700,
  'fmt': '76.73M',
  'longFmt': '76,730,700'},
 'expireDate': {},
 'yield': {},
 'algorithm': None,
 'dividendRate': {},
 'exDividendDate': {'raw': 1580256000, 'fmt': '2020-01-29'},
 'beta': {'raw': 1.232243, 'fmt': '1.23'},
 'circulatingSupply': {},
 'startDate': {},

# Statistics

In [158]:
response=requests.get(url_status.format(stock,stock))
soup= BeautifulSoup(response.text,'html.parser')
pattern= re.compile(r'\s--\sData\s--\s')
script_data=soup.find('script',text=pattern).contents[0]
start =script_data.find("context")-2
json_data=json.loads(script_data[start:-12])

In [98]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()

dict_keys(['defaultKeyStatistics', 'financialsTemplate', 'price', 'financialData', 'quoteType', 'calendarEvents', 'summaryDetail', 'symbol', 'pageViews'])

In [160]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['defaultKeyStatistics']

{'annualHoldingsTurnover': {},
 'enterpriseToRevenue': {'raw': 1.246, 'fmt': '1.25'},
 'beta3Year': {},
 'profitMargins': {'raw': -0.00124, 'fmt': '-0.12%'},
 'enterpriseToEbitda': {'raw': 23.133, 'fmt': '23.13'},
 '52WeekChange': {'raw': -0.045810044, 'fmt': '-4.58%'},
 'morningStarRiskRating': {},
 'forwardEps': {'raw': 1.06, 'fmt': '1.06'},
 'revenueQuarterlyGrowth': {},
 'sharesOutstanding': {'raw': 3907579904,
  'fmt': '3.91B',
  'longFmt': '3,907,579,904'},
 'fundInceptionDate': {},
 'annualReportExpenseRatio': {},
 'totalAssets': {},
 'bookValue': {'raw': 8.326, 'fmt': '8.33'},
 'sharesShort': {'raw': 89221619, 'fmt': '89.22M', 'longFmt': '89,221,619'},
 'sharesPercentSharesOut': {'raw': 0.0224, 'fmt': '2.24%'},
 'fundFamily': None,
 'lastFiscalYearEnd': {'raw': 1577750400, 'fmt': '2019-12-31'},
 'heldPercentInstitutions': {'raw': 0.51794, 'fmt': '51.79%'},
 'netIncomeToCommon': {'raw': -163000000,
  'fmt': '-163M',
  'longFmt': '-163,000,000'},
 'trailingEps': {'raw': -0.041, '

# History of stock data

In [161]:
stock_url='https://query1.finance.yahoo.com/v7/finance/download/F?period1=1573759145&period2=1605381545&interval=1d&events=history&includeAdjustedClose=true'

In [162]:
response=requests.get(stock_url)

In [163]:
response.text

'Date,Open,High,Low,Close,Adj Close,Volume\n2019-11-14,8.850000,8.910000,8.780000,8.790000,8.643010,26527800\n2019-11-15,8.850000,8.960000,8.850000,8.950000,8.800335,26416300\n2019-11-18,9.050000,9.050000,8.890000,8.950000,8.800335,38291400\n2019-11-19,8.990000,9.000000,8.870000,8.900000,8.751170,31168900\n2019-11-20,8.880000,8.890000,8.680000,8.730000,8.584013,38271500\n2019-11-21,8.770000,8.790000,8.670000,8.710000,8.564348,33161400\n2019-11-22,8.800000,8.900000,8.770000,8.890000,8.741339,34966700\n2019-11-25,8.900000,9.010000,8.870000,9.000000,8.849499,30580900\n2019-11-26,8.980000,9.020000,8.910000,9.010000,8.859331,30093800\n2019-11-27,9.030000,9.150000,9.020000,9.100000,8.947826,37396100\n2019-11-29,9.040000,9.100000,9.030000,9.060000,8.908496,13096200\n2019-12-02,9.080000,9.140000,9.000000,9.010000,8.859331,37232700\n2019-12-03,8.950000,8.950000,8.800000,8.890000,8.741339,40653100\n2019-12-04,8.950000,9.030000,8.940000,8.950000,8.800335,29982900\n2019-12-05,8.970000,9.000000,8.8

In [164]:
stock_url='https://query1.finance.yahoo.com/v7/finance/download/F?'
params={
    'period1':'1573759145',
    'period2':'1605381545',
    'interval':'1d',
    'events':'history'
}

In [165]:
params={
    'range':'5y',
    'interval':'1d',
    'events':'history'
}

In [166]:
reponse=requests.get(stock_url.format(stock),params=params)

In [167]:
reponse.text

'Date,Open,High,Low,Close,Adj Close,Volume\n2015-11-16,13.850000,14.080000,13.850000,14.040000,10.643991,22636000\n2015-11-17,14.020000,14.370000,14.010000,14.150000,10.727381,25188600\n2015-11-18,14.230000,14.500000,14.180000,14.460000,10.962399,26250400\n2015-11-19,14.450000,14.630000,14.420000,14.570000,11.045791,23593800\n2015-11-20,14.620000,14.760000,14.540000,14.600000,11.068534,18906200\n2015-11-23,14.600000,14.730000,14.510000,14.550000,11.030630,18273900\n2015-11-24,14.410000,14.630000,14.320000,14.550000,11.030630,21251800\n2015-11-25,14.540000,14.680000,14.520000,14.550000,11.030630,12627000\n2015-11-27,14.540000,14.590000,14.500000,14.530000,11.015468,7760600\n2015-11-30,14.540000,14.550000,14.290000,14.330000,10.863843,27860300\n2015-12-01,14.320000,14.560000,14.290000,14.560000,11.038213,29142200\n2015-12-02,14.600000,14.620000,14.240000,14.260000,10.810778,21398100\n2015-12-03,14.310000,14.340000,13.930000,13.980000,10.598502,29847300\n2015-12-04,14.080000,14.260000,14.

In [168]:
file=StringIO(response.text)
reader =csv.reader(file)
data =list(reader)
for row in data[:5]:
    print(row)

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
['2019-11-14', '8.850000', '8.910000', '8.780000', '8.790000', '8.643010', '26527800']
['2019-11-15', '8.850000', '8.960000', '8.850000', '8.950000', '8.800335', '26416300']
['2019-11-18', '9.050000', '9.050000', '8.890000', '8.950000', '8.800335', '38291400']
['2019-11-19', '8.990000', '9.000000', '8.870000', '8.900000', '8.751170', '31168900']


In [169]:
params={
    'range':'5y',
    'interval':'1w',
    'events':'history'
}

In [170]:
reponse=requests.get(stock_url.format(stock), params=params)

In [171]:
file=StringIO(response.text)
reader =csv.reader(file)
data =list(reader)
for row in data[:5]:
    print(row)

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
['2019-11-14', '8.850000', '8.910000', '8.780000', '8.790000', '8.643010', '26527800']
['2019-11-15', '8.850000', '8.960000', '8.850000', '8.950000', '8.800335', '26416300']
['2019-11-18', '9.050000', '9.050000', '8.890000', '8.950000', '8.800335', '38291400']
['2019-11-19', '8.990000', '9.000000', '8.870000', '8.900000', '8.751170', '31168900']
